In [2]:
import pandas as pd
import numpy as np

# User Based Approach

In [23]:
train = pd.read_csv('./train.csv')
train.head()

,user_id,title,movie_id,rating,release_date,sex,age
0,2592,Top Gun (1986),1101,4,Action|Romance,50,M
1,4318,12 Angry Men (1957),1203,4,Drama,25,M
2,2756,Robocop 2 (1990),2986,2,Action|Crime|Sci-Fi,18,M
3,1706,Modern Times (1936),3462,5,Comedy,25,M
4,4813,Milk Money (1994),276,3,Comedy|Romance,35,F


## Computing relevant items for each user

In [24]:
def get_threshold(user):
    df_aux = train[train['user_id']==user]
    df_aux = df_aux.sort_values(by='rating', ascending=False)
    df_aux.reset_index(inplace = True, drop = True)
    perc_15 =np.trunc(df_aux.shape[0]*15/100)
    thresh = df_aux.loc[perc_15,'rating']
    return float(thresh)


def evaluate_relevance(value,threshold):
    if value>= threshold:
        return 1 
    else:
        return 0

In [29]:
for user in train['user_id'].unique():
    df_aux = train[train['user_id']==user]
    n_movies = df_aux.shape[0]
    thresh = get_threshold(user)
    train.loc[train['user_id'] == user, 'relevance'] = train.loc[train['user_id'] == user,'rating'].apply(lambda x: evaluate_relevance(x, thresh)/n_movies)

In [30]:
train

,user_id,title,movie_id,rating,release_date,sex,age,relevance
0,2592,Top Gun (1986),1101,4,Action|Romance,50,M,0.000000
1,4318,12 Angry Men (1957),1203,4,Drama,25,M,0.000000
2,2756,Robocop 2 (1990),2986,2,Action|Crime|Sci-Fi,18,M,0.000000
3,1706,Modern Times (1936),3462,5,Comedy,25,M,0.005988
4,4813,Milk Money (1994),276,3,Comedy|Romance,35,F,0.000000
...,...,...,...,...,...,...,...,...
800162,59,"Big Chill, The (1983)",2352,4,Comedy|Drama,50,F,0.006024
800163,4458,So I Married an Axe Murderer (1993),543,4,Comedy|Romance|Thriller,25,F,0.003096
800164,1234,Almost Famous (2000),3897,4,Comedy|Drama,18,M,0.000000
800165,4864,"Fish Called Wanda, A (1988)",1079,5,Comedy,18,M,0.038462


In [ ]:
#create a pivot table with users and movies
# use dot product

In [6]:
df_aux = df_aux.sort_values(by='rating', ascending=False)
perc_15 =np.trunc(df_aux.shape[0]*15/100)
df_aux

,user_id,title,movie_id,rating,release_date,sex,age
798327,2592,Raging Bull (1980),1228,5,Drama,50,M
564569,2592,"Sandpiper, The (1965)",3073,5,Drama|Romance,50,M
479541,2592,Son of Flubber (1963),2098,5,Children's|Comedy,50,M
528602,2592,"Bridge on the River Kwai, The (1957)",1250,5,Drama|War,50,M
202705,2592,Patton (1970),1272,5,Drama|War,50,M
...,...,...,...,...,...,...,...
442295,2592,"Truman Show, The (1998)",1682,2,Drama,50,M
587976,2592,Beetlejuice (1988),2174,1,Comedy|Fantasy,50,M
112133,2592,Natural Born Killers (1994),288,1,Action|Thriller,50,M
62356,2592,Things to Do in Denver when You're Dead (1995),81,1,Crime|Drama|Romance,50,M


In [9]:
np.trunc(df_aux.shape[0]*15/100)

52.0

In [ ]:
import numpy as np

In [ ]:
def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

def precision(recommended_items, relevant_items):
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
    
    return precision_score

def recall(recommended_items, relevant_items):  
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]
    
    return recall_score

def AP(recommended_items, relevant_items):
   
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

In [ ]:
#NETFLIX REAL 50.000.000 usuaris and 100.000 items
%autosave 150
%matplotlib inline
import pandas as pd
import numpy as np
import math
import matplotlib.pylab as plt

# Load Data set
u_cols = ['user_id', 'sex', 'age', 'occupation', 'zip_code']
users = pd.read_csv('./data/ml-1m/users.dat', sep='::', names=u_cols)

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('./data/ml-1m/ratings.dat', sep='::', names=r_cols)

# the movies file contains columns indicating the movie's genres
# let's only load the first three columns of the file with usecols
m_cols = ['movie_id', 'title', 'release_date',]
movies = pd.read_csv('./data/ml-1m/movies.dat', sep='::', names=m_cols, usecols=range(3), encoding='latin-1')

# Construcció del DataFrame
data = pd.merge(pd.merge(ratings, users), movies)
data = data[['user_id','title', 'movie_id','rating','release_date','sex','age']]


n_users = data.user_id.nunique()
n_items = data.movie_id.nunique()
print("La BD has "+ str(data.shape[0]) +" ratings")
print("La BD has ", n_users," users")
print("La BD has ", n_items, " movies")
data.head()


In [ ]:
from scipy.stats import pearsonr
from scipy.spatial.distance import euclidean

# Returns a distance-based similarity score for person1 and person2
def SimEuclid(DataFrame,User1,User2,min_common_items=1):
    # GET MOVIES OF USER1
    movies_user1=DataFrame[DataFrame['user_id'] ==User1 ]
    # GET MOVIES OF USER2
    movies_user2=DataFrame[DataFrame['user_id'] ==User2 ]
    
    # FIND SHARED FILMS
    rep=pd.merge(movies_user1 ,movies_user2,on='movie_id')    
    if len(rep)<2:
        return 0
    if(len(rep)<min_common_items):
        return 0
    #return distEuclid(rep['rating_x'],rep['rating_y']) 
    return 1.0/(1.0+euclidean(rep['rating_x'],rep['rating_y'])) 

# Returns a pearsonCorrealation-based similarity score for person1 and person2
def SimPearson(DataFrame,User1,User2,min_common_items=1):
    # GET MOVIES OF USER1
    movies_user1=DataFrame[DataFrame['user_id'] ==User1 ]
    # GET MOVIES OF USER2
    movies_user2=DataFrame[DataFrame['user_id'] ==User2 ]
    
    # FIND SHARED FILMS
    rep=pd.merge(movies_user1 ,movies_user2,on='movie_id',)
    if len(rep)<2:
        return 0    
    if(len(rep)<min_common_items):
        return 0    
    res=pearsonr(rep['rating_x'],rep['rating_y'])[0]
    if(np.isnan(res)):
        return 0
    
    return res

def SimDot()

In [ ]:
from tqdm import tqdm # conda install -y tqdm

class CollaborativeFiltering():
    """ Collaborative filtering using a custom sim(u,u'). """
    
    def __init__(self, similarity=SimPearson):
        """ Constructor """
        self.sim_method=similarity# Gets recommendations for a person by using a weighted average
        self.sim = {}
        
    def fit(self,train):
        """ Prepare data structures for estimation. Similarity matrix for users """
        print("Learning...")
        self.train = train
        allUsers=set(self.train['user_id'])
        
        for person1 in tqdm(allUsers):
            self.sim.setdefault(person1, {})
            a=self.train[self.train['user_id']==person1][['movie_id']]
            data_reduced=pd.merge(self.train,a,on='movie_id') # reduce time complexity
            for person2 in allUsers:
                if person1==person2: continue
                self.sim.setdefault(person2, {})
                if(person1 in self.sim[person2]):continue # since is a simetric matrix
                sim=self.sim_method(data_reduced,person1,person2)
                if(sim<0):
                    self.sim[person1][person2]=0
                    self.sim[person2][person1]=0
                else:
                    self.sim[person1][person2]=sim
                    self.sim[person2][person1]=sim
                
    def predict_score(self, user_id, movie_id):
        totals={}
        movie_users=self.train[self.train['movie_id'] ==movie_id]
        rating_num=0.0
        rating_den=0.0
        allUsers=set(movie_users['user_id'])
        for other in allUsers:
            if user_id==other: continue 
            rating_num += self.sim[user_id][other] * float(movie_users[movie_users['user_id']==other]['rating'])
            rating_den += self.sim[user_id][other]
        if rating_den==0: 
            if self.train.rating[self.train['movie_id']==movie_id].mean()>0:
                # return the mean movie rating if there is no similar for the computation
                return self.train.rating[self.train['movie_id']==movie_id].mean()
            else:
                # else return mean user rating 
                return self.train.rating[self.train['user_id']==user_id].mean()
        return rating_num/rating_den
    
    def predict_top(self, user_id, at=5, remove_seen=True):
        '''Given a user_id predict its top AT items'''
        seen_items = self.train[self.train.user_id==user_id].movie_id.values
        unseen_items = set(self.train.movie_id.values) - set(seen_items)

        predictions = [(item_id,self.predict_score(user_id,item_id)) for item_id in unseen_items]

        sorted_predictions = sorted(predictions, key=lambda x: x[1],reverse = True)[:at]
        return [i[0] for i in sorted_predictions]
    